# Importing all the required libraries

In [1]:
# We are importing files here
import pandas as pd
import nltk
import re
from nltk.corpus import wordnet
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from bs4 import BeautifulSoup
from collections import Counter
import csv

# Importing all the Data

In [2]:
# We are importing here our data

# Projects Details
projects = pd.read_excel(r'F:\Downloads\round1.xlsx')
projects.head()

# Training Data
training = pd.read_excel(r'F:\Downloads\round1.xlsx', sheet_name='Train')
training.head()

# Test Data
test = pd.read_excel(r'F:\Downloads\round1.xlsx', sheet_name='Test')
test.head()

submission = pd.read_excel(r'F:\Downloads\round1.xlsx', sheet_name='Solution_submission')
submission.head()

,Submission
0,p10
1,p100
2,p107
3,p115
4,p121


# This here does all the data cleaning work

In [3]:
# this is to make all the required data to lowercase so as to make the cosine similarity work evenly
training['skills'] = training['skills'].str.lower()
projects['required_skills'] = projects['required_skills'].str.lower()
projects['description'] = projects['description'].str.lower()


# remove the html p tag from project description using Beautiful Soup
for ind in projects.index:
    projects["description"][ind] = BeautifulSoup(projects["description"][ind] , "lxml").text
    
# Get the most common words from the projects description which are important and keep only those words
list1 = [] # this will store the project descriptions
s = " "
for ind in projects.index:
    list1.append(projects["description"][ind])

s = s.join(list1) # this is a string of all the project description into one

to_keep = ['data','frontend','backend','visualization','video','prediction','application','business','cloud','analytics','learning','applications','technology','mobile','machine','model','digital','tools','api','system','analysis','design','automation','database','ai','app','iot','aws','ar','models','android','azure','network','ml','microservices','developer','ui','processing','algorithms','python','image','dashboard','virtual','nlp','programming']
text_tokens = word_tokenize(s)
tokens_without_sw = [word for word in text_tokens if word in to_keep] 

count = Counter(tokens_without_sw) # counts the words
most_occur = count.most_common(400) # prints most commons words




# ALSO CLEAN THE TEST DATASET
test["skills"] = test["skills"].str.lower()

In [4]:
#Drop any row having value as null
training = training.dropna(how='any', axis=0)

# ALSO DROP ANY ROW HAVING VALUE NULL
test = test.dropna(how='any', axis=0)

# Checking for few things to work

In [5]:
# Checking: This is getting the required index from the projects dataframe
index = projects[projects['id']=='p5'].index.values.astype(int)[0]

# Checking for domain working
training["skills"] = training["skills"].str.replace("python","python programming analytics") # its working
training["skills"] = training["skills"].str.replace(" java "," java programming ")
training["skills"] = training["skills"].str.replace(" c "," c programming ")
training["skills"] = training["skills"].str.replace(" r "," r analytics ")
training["skills"] = training["skills"].str.replace(" strucres "," structures ")
training["skills"] = training["skills"].str.replace(" node.js "," node.js web development ")
training["skills"] = training["skills"].str.replace("kubernetes", " kubernetes microservices ")
training["skills"] = training["skills"].str.replace("sql", " sql database ")
training["skills"] = training["skills"].str.replace("selenium", " selenium automation ")
training["skills"] = training["skills"].str.replace("flutter", " flutter mobile application ")

projects["description"] = projects["description"].str.replace(" angular ", " angular javascript web ")
projects["required_skills"] = projects["required_skills"].str.replace(" angular ", " angular javascript web ")
projects["description"] = projects["description"].str.replace("analytical", " analytics ")
projects["description"] = projects["description"].str.replace(" analyses ", " analytics ")
projects["required_skills"] = projects["required_skills"].str.replace("analytical", " analytics ")
projects["required_skills"] = projects["required_skills"].str.replace(" analyses ", " analytics ")
projects["required_skills"] = projects["required_skills"].str.replace("java/python", " java python programming ")
projects["required_skills"] = projects["required_skills"].str.replace("angular", " angular web development ")
projects["required_skills"] = projects["required_skills"].str.replace("coding", " programming ")
projects["required_skills"] = projects["required_skills"].str.replace("flow", "")
projects["required_skills"] = projects["required_skills"].str.replace("tensor", " tensorflow ")
projects["required_skills"] = projects["required_skills"].str.replace("mean/mern", " mongodb express angular.js node.js react.js ")

In [6]:
# ALSO DO THE ABOVE PROCESS FOR TEST DATASET
test["skills"] = test["skills"].str.replace("python","python programming analytics") # its working
test["skills"] = test["skills"].str.replace(" java "," java programming ")
test["skills"] = test["skills"].str.replace(" c "," c programming ")
test["skills"] = test["skills"].str.replace(" r "," r analytics ")
test["skills"] = test["skills"].str.replace(" strucres "," structures ")
test["skills"] = test["skills"].str.replace(" node.js "," node.js web development ")
test["skills"] = test["skills"].str.replace("kubernetes", " kubernetes microservices ")
test["skills"] = test["skills"].str.replace("sql", " sql database ")
test["skills"] = test["skills"].str.replace("selenium", " selenium automation ")
test["skills"] = test["skills"].str.replace("flutter", " flutter mobile application ")

# Generalising the above whole process for each data - TRAINING

In [7]:
# Getting the total similarity score by iterating over the trainings dataframe
total_similarity_score = 0
total_selected_all = 0
total_selected_s = 0
for ind in training.index: 
    # Store the skills data in X variable
    if (training['evaluation_status'][ind] == 'SELECTED'):
        
        total_selected_all = total_selected_all + 1
        
        X = training['skills'][ind]
    
        # store the project id
        project_id = training['internship_id'][ind]
        
        # get the index in the projects data frame from the above project_id
        index = projects[projects['id']==project_id].index.values.astype(int)[0]

        # Store the project required skills data in Y variable
        Y = projects['required_skills'][index]
        
        # Store the project details data in Z variable
        Z = projects['description'][index]

        # Do the work done for the single data check
        X_list = word_tokenize(X)  
        Y_list = word_tokenize(Y) 
        Z_list = word_tokenize(Z)
        
        # To keep only those words in Z_list which are present in the list to_keep
        Z_list_new = [w for w in Z_list if w in to_keep]
        
    
        
        # Extend the project keywords list with project description keywords
        Y_list.extend(Z_list_new)
        
        # getting stop words
        sw = stopwords.words('english')  
        l1 =[];l2 =[] 
        # appending some words from our side so as to clean the irrelevant data which will increase the cosine similarity score
        sw.append('self')
        sw.append(',')
        sw.append('AND')
        sw.append('techniques')
        sw.append('&')
        sw.append('knowledge')
        sw.append('computer')
        sw.append('#')
        sw.append('things')
        sw.append('good')
        sw.append('like')
        sw.append('tools')
        sw.append('exposure')
        sw.append('basic')
        sw.append('basics')
        sw.append('studio')
        sw.append('positive')
        sw.append('attitude')
        sw.append('6+')
        sw.append('3')
        sw.append('pro')
        sw.append('handons')
        sw.append('experience')
        sw.append('.')
        sw.append('skills')
        sw.append('-')
        sw.append("''")
        sw.append('understanding')
        sw.append('understaning')
        sw.append('concepts')
        sw.append('language')
        sw.append('7')
        sw.append('3.0')
        sw.append('intererst')
        sw.append('new')
        sw.append('building')
        sw.append('towards')
        sw.append('(')
        sw.append(')')
        sw.append('solving')
        sw.append('4')
        sw.append('expertise')
        sw.append('libraries')
        sw.append('illustrator')
        sw.append('advanced')
        sw.append('7+')
        sw.append('..')
        sw.append('/')
        sw.append('advantage')
        sw.append('added')
        sw.append('domain')
        sw.append('competitive')
        sw.append('problem')
        sw.append('formulation')
        sw.append('system')
        sw.append('applications')
        sw.append('expereicne')
        sw.append('hunting')
        sw.append('bounty')
        sw.append('bug')
        sw.append('configuration')
        sw.append('vulnerability')
        sw.append('information')
        sw.append('assessment')
        
        X_set = {w for w in X_list if not w in sw}  
        Y_set = {w for w in Y_list if not w in sw} 
        
        
        
        
        
        rvector = X_set.union(Y_set)  
        for w in rvector: 
            if w in X_set: l1.append(1) # create a vector 
            else: l1.append(0) 
            if w in Y_set: l2.append(1) 
            else: l2.append(0) 
        c = 0
        for i in range(len(rvector)): 
            c+= l1[i]*l2[i] 
        cosine = c / float((sum(l1)*sum(l2))**0.5) 
        
        if(cosine != 0.0):
            total_selected_s = total_selected_s + 1
        
        
        total_similarity_score = total_similarity_score + cosine


# Get the threshold  values for prediction of candidates from test data

In [8]:
# Divide the total_similarity_score by total training data for getting the threshold


# this is for all the selected candidates though they dont match any skills
threshold_min = total_similarity_score / total_selected_all
print("Minimum Threshold: ", threshold_min)

# this is for candidates whose skils have atleast contributed to the similarity score
threshold_max = total_similarity_score / total_selected_s
print("Maximum Threshold: ", threshold_max)

Minimum Threshold:  0.1687196018278202
Maximum Threshold:  0.22780191726933124


# Code to get the candidates for only 1st project data from the - TEST DATASET

In [9]:
selected_candidates_list = []
for ind in test.index:
    X = test['skills'][ind]
    Y = projects['required_skills'][0]
    Z = projects['description'][0]
    
    X_list = word_tokenize(X)  
    Y_list = word_tokenize(Y) 
    Z_list = word_tokenize(Z)
        
    Z_list_new = [w for w in Z_list if w in to_keep]
    Y_list.extend(Z_list_new)
    
    sw = stopwords.words('english')  
    l1 =[];l2 =[] 
    sw.append('self')
    sw.append(',')
    sw.append('AND')
    sw.append('techniques')
    sw.append('&')
    sw.append('knowledge')
    sw.append('computer')
    sw.append('#')
    sw.append('things')
    sw.append('good')
    sw.append('like')
    sw.append('tools')
    sw.append('exposure')
    sw.append('basic')
    sw.append('basics')
    sw.append('studio')
    sw.append('positive')
    sw.append('attitude')
    sw.append('6+')
    sw.append('3')
    sw.append('pro')
    sw.append('handons')
    sw.append('experience')
    sw.append('.')
    sw.append('skills')
    sw.append('-')
    sw.append("''")
    sw.append('understanding')
    sw.append('understaning')
    sw.append('concepts')
    sw.append('language')
    sw.append('7')
    sw.append('3.0')
    sw.append('intererst')
    sw.append('new')
    sw.append('building')
    sw.append('towards')
    sw.append('(')
    sw.append(')')
    sw.append('solving')
    sw.append('4')
    sw.append('expertise')
    sw.append('libraries')
    sw.append('illustrator')
    sw.append('advanced')
    sw.append('7+')
    sw.append('..')
    sw.append('/')
    sw.append('advantage')
    sw.append('added')
    sw.append('domain')
    sw.append('competitive')
    sw.append('problem')
    sw.append('formulation')
    sw.append('system')
    sw.append('applications')
    sw.append('expereicne')
    sw.append('hunting')
    sw.append('bounty')
    sw.append('bug')
    sw.append('configuration')
    sw.append('vulnerability')
    sw.append('information')
    sw.append('assessment')
    
    
    X_set = {w for w in X_list if not w in sw}  
    Y_set = {w for w in Y_list if not w in sw} 
        
               
    rvector = X_set.union(Y_set)  
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector 
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0
    for i in range(len(rvector)): 
        c+= l1[i]*l2[i] 
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    
    if(cosine > threshold_min):
        
        def takeSecond(elem):
            return elem[1]
        
        t = (test["candidate_id"][ind], cosine)
        selected_candidates_list.append(t)
    

selected_candidates_list.sort(key=takeSecond, reverse=True)

l = 0

if(len(selected_candidates_list) >= 5):
    l = 5
else:
    l = len(selected_candidates_list)
   

s = ','
top = []
top.append(projects['id'][10])
for i in range(l):
    top.append(selected_candidates_list[i][0])
    
s = s.join(top)
print(s)

p15,c432


# Lets generalise the above code for all the projects mentioned in submission file

In [10]:
final = []
for ind1 in submission.index:
    
    selected_candidates_list = []
    
    for ind in test.index:
        X = test['skills'][ind]
        
        # store the project id
        project_id = submission['Submission '][ind1]
        
        # get the index in the projects data frame from the above project_id
        index = projects[projects['id']==project_id].index.values.astype(int)[0]

        # Store the project required skills data in Y variable
        Y = projects['required_skills'][index]

        Z = projects['description'][index]

        X_list = word_tokenize(X)  
        Y_list = word_tokenize(Y) 
        Z_list = word_tokenize(Z)

        Z_list_new = [w for w in Z_list if w in to_keep]
        Y_list.extend(Z_list_new)

        sw = stopwords.words('english')  
        l1 =[];l2 =[] 
        sw.append('self')
        sw.append(',')
        sw.append('AND')
        sw.append('techniques')
        sw.append('&')
        sw.append('knowledge')
        sw.append('computer')
        sw.append('#')
        sw.append('things')
        sw.append('good')
        sw.append('like')
        sw.append('tools')
        sw.append('exposure')
        sw.append('basic')
        sw.append('basics')
        sw.append('studio')
        sw.append('positive')
        sw.append('attitude')
        sw.append('6+')
        sw.append('3')
        sw.append('pro')
        sw.append('handons')
        sw.append('experience')
        sw.append('.')
        sw.append('skills')
        sw.append('-')
        sw.append("''")
        sw.append('understanding')
        sw.append('understaning')
        sw.append('concepts')
        sw.append('language')
        sw.append('7')
        sw.append('3.0')
        sw.append('intererst')
        sw.append('new')
        sw.append('building')
        sw.append('towards')
        sw.append('(')
        sw.append(')')
        sw.append('solving')
        sw.append('4')
        sw.append('expertise')
        sw.append('libraries')
        sw.append('illustrator')
        sw.append('advanced')
        sw.append('7+')
        sw.append('..')
        sw.append('/')
        sw.append('advantage')
        sw.append('added')
        sw.append('domain')
        sw.append('competitive')
        sw.append('problem')
        sw.append('formulation')
        sw.append('system')
        sw.append('applications')
        sw.append('expereicne')
        sw.append('hunting')
        sw.append('bounty')
        sw.append('bug')
        sw.append('configuration')
        sw.append('vulnerability')
        sw.append('information')
        sw.append('assessment')


        X_set = {w for w in X_list if not w in sw}  
        Y_set = {w for w in Y_list if not w in sw} 


        rvector = X_set.union(Y_set)  
        for w in rvector: 
            if w in X_set: l1.append(1) # create a vector 
            else: l1.append(0) 
            if w in Y_set: l2.append(1) 
            else: l2.append(0) 
        c = 0
        for i in range(len(rvector)): 
            c+= l1[i]*l2[i] 
        cosine = c / float((sum(l1)*sum(l2))**0.5)

        if(cosine > threshold_min):

            def takeSecond(elem):
                return elem[1]

            t = (test["candidate_id"][ind], cosine)
            selected_candidates_list.append(t)


    selected_candidates_list.sort(key=takeSecond, reverse=True)

    l = 0

    if(len(selected_candidates_list) >= 5):
        l = 5
    else:
        l = len(selected_candidates_list)


    s = ','
    top = []
    top.append(projects['id'][10])
    for i in range(l):
        top.append(selected_candidates_list[i][0])

    s = s.join(top)
    submission['Submission '][ind1] = s
    submission.to_csv('Solution_submission.csv', index=False)
